## Libraries

In [1]:
!pip install datasets==2.15

ERROR: Could not find a version that satisfies the requirement datasets==2.15 (from versions: none)
ERROR: No matching distribution found for datasets==2.15


In [2]:
# Importing necessary libraries
import nltk
import numpy as np # linear algebra
import lightgbm as lgb
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier,GradientBoostingClassifier,BaggingClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score
import matplotlib.pyplot as plt
from sklearn.metrics import cohen_kappa_score
from lightgbm import log_evaluation, early_stopping
from sklearn.linear_model import SGDClassifier
import polars as pl
import torch
from IPython.display import display
from datasets import Dataset,DatasetDict
from transformers import AutoTokenizer
from transformers import AutoModel

nltk.download('wordnet')

[nltk_data] Error loading wordnet: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>


False

### Reading the Data

In [3]:
columns = [  
    (
        pl.col("full_text").str.split(by="\n\n").alias("paragraph")
    ),
]
PATH = "/kaggle/input/learning-agency-lab-automated-essay-scoring-2/"
# 载入训练集和测试集，同时对full_text数据使用\n\n字符分割为列表，重命名为paragraph
# Load training and testing sets, while using \ n \ n character segmentation to list and renaming to paragraph for full_text data
train = pl.read_csv(PATH + "train.csv").with_columns(columns)

# for test only
#train = train.sample(200)
test = pl.read_csv(PATH + "test.csv").with_columns(columns)
# 显示训练集中的第一个样本数据
# Display the first sample data in the training set
train.head(1)

essay_id,full_text,score,paragraph
str,str,i64,list[str]
"""000d118""","""Many people ha…",3,"[""Many people have car where they live. The thing they don't know is that when you use a car alot of thing can happen like you can get in accidet or the smoke that the car has is bad to breath on if someone is walk but in VAUBAN,Germany they dont have that proble because 70 percent of vauban's families do not own cars,and 57 percent sold a car to move there. Street parkig ,driveways and home garages are forbidden on the outskirts of freiburd that near the French and Swiss borders. You probaly won't see a car in Vauban's streets because they are completely ""car free"" but If some that lives in VAUBAN that owns a car ownership is allowed,but there are only two places that you can park a large garages at the edge of the development,where a car owner buys a space but it not cheap to buy one they sell the space for you car for $40,000 along with a home. The vauban people completed this in 2006 ,they said that this an example of a growing trend in Europe,The untile states and some where else are suburban life from auto use this is called ""smart planning"". The current efforts to drastically reduce greenhouse gas emissions from tailes the passengee cars are responsible for 12 percent of greenhouse gas emissions in Europe and up to 50 percent in some car intensive in the United States. I honeslty think that good idea that they did that is Vaudan because that makes cities denser and better for walking and in VAUBAN there are 5,500 residents within a rectangular square mile. In the artical David Gold berg said that ""All of our development since World war 2 has been centered on the cars,and that will have to change"" and i think that was very true what David Gold said because alot thing we need cars to do we can go anyway were with out cars beacuse some people are a very lazy to walk to place thats why they alot of people use car and i think that it was a good idea that that they did that in VAUBAN so people can see how we really don't need car to go to place from place because we can walk from were we need to go or we can ride bycles with out the use of a car. It good that they are doing that if you thik about your help the earth in way and thats a very good thing to. In the United states ,the Environmental protection Agency is promoting what is called ""car reduced""communtunties,and the legislators are starting to act,if cautiously. Maany experts expect pubic transport serving suburbs to play a much larger role in a new six years federal transportation bill to approved this year. In previous bill,80 percent of appropriations have by law gone to highways and only 20 percent to other transports. There many good reason why they should do this. ""]"


### Preprocessing 

In [4]:
def removeHTML(x):
    html=re.compile(r'<.*?>')
    return html.sub(r'',x)
def dataPreprocessing(x):
    # 将单词转化为小写
    # Convert words to lowercase
    x = x.lower()
    # Remove HTML
    # 移除html
    x = removeHTML(x)
    # 删除以@作为首字母的字符串
    # Delete strings starting with @
    x = re.sub("@\w+", '',x)
    # 删除数字
    # Delete Numbers
    x = re.sub("'\d+", '',x) # can delete it
    x = re.sub("\d+", '',x)
    # 删除网址
    # Delete URL
    x = re.sub("http\w+", '',x)
    # 将连续空白符替换为一个空格字符
    # Replace consecutive empty spaces with a single space character
    x = re.sub(r"\s+", " ", x)
    # 替换连续的句号和逗号为一个
    # Replace consecutive commas and periods with one comma and period character
    x = re.sub(r"\.+", ".", x)
    x = re.sub(r"\,+", ",", x)
    # 去除开头结尾的空白符
    # Remove empty characters at the beginning and end
    x = x.strip()
    return x

### Feature extraction

In [5]:
# convert the df to dataset 
train = train.with_columns(pl.col('full_text').map_elements(dataPreprocessing,return_dtype=str))
test = test.with_columns(pl.col('full_text').map_elements(dataPreprocessing,return_dtype=str))

ds_train = Dataset.from_pandas(train.to_pandas())
ds_test = Dataset.from_pandas(test.to_pandas())

ds = DatasetDict({"train": ds_train, "test": ds_test})

# Print the dataset dictionary keys and sizes
for name, dataset in ds.items():
    print(f"{name} size:", len(dataset))


train size: 17307
test size: 3


In [6]:
# tokenize 
model_ckpt = "/kaggle/input/deberta-v3/pytorch/base/2"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

def tokenize(batch):
    return tokenizer(batch["full_text"], padding=True)

dst = ds.map(tokenize, batched=True ,batch_size = 64)
print(dst)

/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/17307 [00:00<?, ? examples/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['essay_id', 'full_text', 'score', 'paragraph', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 17307
    })
    test: Dataset({
        features: ['essay_id', 'full_text', 'paragraph', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3
    })
})


In [7]:
# load the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModel.from_pretrained(model_ckpt).to(device)

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [8]:
# set format to tensors, ( needed for the model Feedforward )
dst.set_format("torch",columns=["input_ids", "attention_mask"])

# remove unwanted features
columns_to_remove = ['full_text', 'paragraph']
dstr = dst.remove_columns(columns_to_remove)

In [9]:
# extract the features
def extract_hidden_states(batch):
    # Place model inputs on the GPU
    inputs = {k:v.to(device) for k,v in batch.items() 
              if k in tokenizer.model_input_names}
    # Extract last hidden states
    with torch.no_grad():
        last_hidden_state = model(**inputs).last_hidden_state
    # Return vector for [CLS] token
    return {"hidden_state": last_hidden_state[:,0].cpu().numpy()}

# prepare the vectors
dstr_hidden = dstr.map(extract_hidden_states,batched=True,batch_size = 2)

Map:   0%|          | 0/17307 [00:00<?, ? examples/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [10]:
features_df = pd.DataFrame(dstr_hidden['train']['hidden_state'].numpy())
features_df.head(3)

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.017458,0.088095,0.011389,-0.045140,-0.047742,-0.028921,-0.013408,-0.042997,0.018397,0.074320,...,-0.193040,-0.079157,-0.032183,-0.105401,-0.112349,-0.130857,0.013521,-0.080458,0.029315,0.023254
1,0.034739,0.088562,-0.007047,-0.018241,-0.048013,-0.054728,-0.021245,-0.031907,0.043690,0.045048,...,-0.176704,-0.056483,-0.079937,-0.057016,-0.088996,-0.134486,0.073281,-0.043578,0.071187,0.048367
2,-0.000643,0.047554,-0.037517,0.007603,-0.077885,-0.095791,-0.015752,0.000893,-0.018395,0.009504,...,-0.162281,-0.049000,-0.040095,-0.096680,-0.078249,-0.108106,0.082873,-0.062704,0.076290,0.076560


In [11]:
features_df['essay_id'] = dstr_hidden['train']['essay_id']
features_df.head(3)

,0,1,2,3,4,5,6,7,8,9,...,759,760,761,762,763,764,765,766,767,essay_id
0,0.017458,0.088095,0.011389,-0.045140,-0.047742,-0.028921,-0.013408,-0.042997,0.018397,0.074320,...,-0.079157,-0.032183,-0.105401,-0.112349,-0.130857,0.013521,-0.080458,0.029315,0.023254,000d118
1,0.034739,0.088562,-0.007047,-0.018241,-0.048013,-0.054728,-0.021245,-0.031907,0.043690,0.045048,...,-0.056483,-0.079937,-0.057016,-0.088996,-0.134486,0.073281,-0.043578,0.071187,0.048367,000fe60
2,-0.000643,0.047554,-0.037517,0.007603,-0.077885,-0.095791,-0.015752,0.000893,-0.018395,0.009504,...,-0.049000,-0.040095,-0.096680,-0.078249,-0.108106,0.082873,-0.062704,0.076290,0.076560,001ab80


In [12]:
features_df.to_csv("features_deberta.csv",index=False)